In [2]:
import mysql.connector
import json
import uuid

In [3]:
# Connection parameters
host = 'localhost'
user = 'root'
password = 'Ncgncg1102@'
database = 'stock_prediction'

In [4]:
data="C:\\Users\\GIANG\DAC_Project\\Project_python\\2024-dn-python-stockprediction\\crawlData\\data.json"
with open(data,'r') as file:
  data=json.load(file)


<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\GIANG\AppData\Local\Temp\ipykernel_19656\4237248046.py:1: SyntaxWarning: invalid escape sequence '\D'
  data="C:\\Users\\GIANG\DAC_Project\\Project_python\\2024-dn-python-stockprediction\\crawlData\\data.json"


In [5]:
symboys=list(data.keys())
name_symboys=["Ngân hàng TMCP Á Châu","Ngân hàng TMCP Đầu tư và Phát triển Việt Nam","Ngân hàng TMCP Công Thương Việt Nam","Ngân hàng TMCP Xuất nhập khẩu Việt Nam","Ngân hàng TMCP Phát triển TP.HCM","Ngân hàng TMCP Quân Đội","Ngân hàng TMCP Hàng hải Việt Nam","Ngân hàng TMCP Phương Đông","Ngân hàng TMCP Sài Gòn – Hà Nội","Ngân hàng TMCP Đông Nam Á","Ngân hàng TMCP Sài Gòn Thương Tín","Ngân hàng TMCP Kỹ thương Việt Nam","Ngân hàng TMCP Tiên Phong","Ngân hàng TMCP Ngoại Thương Việt Nam","Ngân hàng TMCP Quốc tế Việt Nam","Ngân hàng TMCP Việt Nam Thịnh Vượng","Ngân hàng TMCP Quốc Dân","Ngân hàng TMCP Bắc Á"]

In [6]:
stocks=[]
for symboy in symboys:
  stocks.append(list(data[symboy].values()))
previous_close_prices=[]
for stock in stocks:
      previous_close_prices.append(eval(stock[0])["close"])


In [54]:
# data_inserts=[]
# for i in range(len(stocks)):
#     data_inserts.append((str(uuid.uuid4()),symboys[i],name_symboys[i],float(previous_close_prices[i].replace(',', ''))))

In [55]:
# try:
#     conn = mysql.connector.connect(
#         host=host,
#         user=user,
#         password=password,
#         database=database
#     )

#     if conn.is_connected():
#         print("Connected to MySQL")
        
#         cursor = conn.cursor()
#         insert_query = "INSERT INTO stocklist (stockid, symboy, company_name,previous_close_price) VALUES (%s, %s, %s,%s)"

#         cursor.executemany(insert_query, data_inserts)
#         conn.commit()

#         cursor.close()
#         conn.close()

# except mysql.connector.Error as e:
#     print(f"Error: {e}")
# finally:
#     # Make sure to close the connection in case of an exception
#     if conn.is_connected():
#         cursor.close()
#         conn.close()
#         print("MySQL connection closed")

Connected to MySQL
Error: 1062 (23000): Duplicate entry 'ACB' for key 'stocklist.symboy'
MySQL connection closed


In [1]:
data_history=[]
for i in range(len(symboys)):
    data_history.append()

SyntaxError: expected ':' (2659166738.py, line 1)

In [63]:
date=list(data[symboys[0]].keys())

In [64]:
date

['06/02/2024',
 '05/02/2024',
 '02/02/2024',
 '01/02/2024',
 '31/01/2024',
 '30/01/2024',
 '29/01/2024',
 '26/01/2024',
 '25/01/2024',
 '24/01/2024',
 '23/01/2024',
 '22/01/2024',
 '19/01/2024',
 '18/01/2024',
 '17/01/2024',
 '16/01/2024',
 '15/01/2024',
 '12/01/2024',
 '11/01/2024',
 '10/01/2024',
 '09/01/2024',
 '08/01/2024',
 '05/01/2024',
 '04/01/2024',
 '03/01/2024',
 '02/01/2024',
 '29/12/2023',
 '28/12/2023',
 '27/12/2023',
 '26/12/2023',
 '25/12/2023',
 '22/12/2023',
 '21/12/2023',
 '20/12/2023',
 '19/12/2023',
 '18/12/2023',
 '15/12/2023',
 '14/12/2023',
 '13/12/2023',
 '12/12/2023',
 '11/12/2023',
 '08/12/2023',
 '07/12/2023',
 '06/12/2023',
 '05/12/2023',
 '04/12/2023',
 '01/12/2023',
 '30/11/2023',
 '29/11/2023',
 '28/11/2023',
 '27/11/2023',
 '24/11/2023',
 '23/11/2023',
 '22/11/2023',
 '21/11/2023',
 '20/11/2023',
 '17/11/2023',
 '16/11/2023',
 '15/11/2023',
 '14/11/2023',
 '13/11/2023',
 '10/11/2023',
 '09/11/2023',
 '08/11/2023',
 '07/11/2023',
 '06/11/2023',
 '03/11/20

In [57]:
symboys[0]

'ACB'